### Imports

In [7]:
import os
import pandas as pd

import plotly.graph_objects as go
import plotly.io as pio


from log_parser import *
from odrive_utils import *
from figures import *


### Get Logs

In [8]:
# config
log_dir = "logs"
generate_html = True
offline = False
show_figures = False
min_log_size_kb = 1
csv_logs = False
dark_theme = True

In [9]:
# get paths
if csv_logs:
    log_ext = "txt"
    logParserFunc = parseCSVFile
else:
    log_ext = "bin"
    logParserFunc = parseBinaryFile
if dark_theme:
    pio.templates.default = "plotly_dark"
else:
    pio.templates.default = "plotly_white"


paths = getLogsByExtension(log_dir, log_ext)
paths = filterFilesBySize(paths, min_log_size_kb)

In [10]:
# create dataframes
dfs = []

for path in paths:
    _, df = logParserFunc(path)
    if df is None:
        continue
    postProcessDataframe(df)
    df.filename = os.path.basename(path)
    addNormalizedColumns(df)
    dfs.append(df)

### Create per Log Graphs

In [11]:
# create graphs
figure_funcs = [
    getRPMFigure,
    getRPMAndActuatorFigure,
    getVehicleSpeedFigure,
    getShiftRatioAndAcuatorFigure,
    getShiftRatioFigure,
    getVelocityCommandFigure,
    getShadowCountFigure,
    getEngineVsWheelFigure,
    getEverythingFigure,
]

all_figs_by_log = []

for df in dfs:
    figs = []
    for figure_func in figure_funcs:
        figs.append(figure_func(df))
    all_figs_by_log.append(figs)

if generate_html:
    for path, figs_by_log in zip(paths, all_figs_by_log):
        filename_without_ext = os.path.splitext(os.path.basename(path))[0]
        html_path = f"graphs/{filename_without_ext}.html"
        figuresToHTML(figs_by_log, html_path, offline=offline, dark_theme=dark_theme)


In [12]:
import ipywidgets as widgets
from IPython.display import clear_output

show_figures = True
if show_figures:
    dropdown = widgets.Dropdown(options=paths, value=paths[-1])

    def onDropdownChange(change):
        if change["type"] == "change" and change["name"] == "value":
            clear_output()
            idx = paths.index(change["new"])
            printErrorTimes(dfs[idx], ErrorType.MOTOR)
            display(dropdown)
            for fig in all_figs_by_log[idx]:
                fig.show()

    idx = -1
    dropdown.observe(onDropdownChange)
    printErrorTimes(dfs[idx], ErrorType.MOTOR)
    display(dropdown)
    for fig in all_figs_by_log[idx]:
        fig.show()

Motor Errors (10.04):
  NONE


Dropdown(index=7, options=('logs/log_2023-04-22_12-26-00.bin', 'logs/log_2023-04-22_12-26-33.bin', 'logs/log_2…

### Create Group Graphs (By Graph Type)

In [ ]:
"""
figure_names_and_funcs = [
    ("rpm", getRPMFigure),
    ("rpm-and-actuator", getRPMAndActuatorFigure),
    ("vehicle-speed", getVehicleSpeedFigure),
    ("shift-ratio-and-acuator", getShiftRatioAndAcuatorFigure),
    ("velocity-command", getVelocityCommandFigure),
    ("shadow-count", getShadowCountFigure),
    ("shift-ratio", getShiftRatioFigure),
    ("engine-vs-wheel", getEngineVsWheelFigure),
]
all_figs_by_type = []
for name, func in figure_names_and_funcs:
    figs_by_type = []
    for df in dfs:
        figs_by_type.append(func(df))
    all_figs_by_type.append((name, figs_by_type))

if generate_html:
    for figs_by_type in all_figs_by_type:
        filename_without_ext = os.path.splitext(os.path.basename(path))[0]
        html_path = f"graphs/all-{figs_by_type[0]}.html"
        figuresToHTML(figs_by_type[1], html_path, offline=offline)
"""